In [3]:
# import libraries
import os
import re
import pathlib
from glob import glob

import matplotlib.pyplot as plt
import earthaccess
import xrspatial
import geopandas as gpd
import rioxarray as rxr
import rioxarray.merge as rxrmerge

c:\Users\moenc\miniconda3\envs\earth-analytics-python\Lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [1]:
%store -r curl_gdf shey_gdf c_soil_url_list p_soil_url_list

In [27]:
# build project and elevation directories

data_dir = os.path.join(
    pathlib.Path.home(),
    'earth-analytics',
    'data'
)
project_dir = os.path.join(data_dir, 'habitat_suitability')
elevation_dir = os.path.join(data_dir, 'srtm')

os.makedirs(elevation_dir, exist_ok=True)
data_dir

'C:\\Users\\moenc\\earth-analytics\\data'

In [5]:
# login to earthaccess
earthaccess.login(strategy="interactive", persist=True)

In [6]:
# search for the appropriate DEM

datasets = earthaccess.search_datasets(keyword='SRTM DEM', count=11)
for dataset in datasets:
    print(dataset['umm']['ShortName'], dataset['umm']['EntryTitle'])

NASADEM_SHHP NASADEM SRTM-only Height and Height Precision Mosaic Global 1 arc second V001
NASADEM_SIM NASADEM SRTM Image Mosaic Global 1 arc second V001
NASADEM_SSP NASADEM SRTM Subswath Global 1 arc second V001
C_Pools_Fluxes_CONUS_1837 CMS: Terrestrial Carbon Stocks, Emissions, and Fluxes for Conterminous US, 2001-2016
SRTMGL1 NASA Shuttle Radar Topography Mission Global 1 arc second V003
GEDI01_B GEDI L1B Geolocated Waveform Data Global Footprint Level V002
GEDI02_B GEDI L2B Canopy Cover and Vertical Profile Metrics Data Global Footprint Level V002
NASADEM_HGT NASADEM Merged DEM Global 1 arc second V001
SRTMGL3 NASA Shuttle Radar Topography Mission Global 3 arc second V003
SRTMGL1_NC NASA Shuttle Radar Topography Mission Global 1 arc second NetCDF V003
SRTMGL30 NASA Shuttle Radar Topography Mission Global 30 arc second V002


In [7]:
curl_gdf.bounds

,minx,miny,maxx,maxy
16,-112.869628,42.029103,-112.522562,42.331612


In [8]:
shey_gdf.bounds

,minx,miny,maxx,maxy
3,-97.468801,46.095335,-96.938539,46.572034


In [9]:
# xmin, ymin, xmax, ymax = shey_gdf.total_bounds
bounds = tuple(shey_gdf.total_bounds)
srtm_p_results = earthaccess.search_data(
    short_name = "SRTMGL1",
    bounding_box = bounds
)
print(srtm_p_results)  # See if any results are found

[Collection: {'ShortName': 'SRTMGL1', 'Version': '003'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -97.00027778, 'EastBoundingCoordinate': -95.99972222, 'NorthBoundingCoordinate': 47.00027778, 'SouthBoundingCoordinate': 45.99972222}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2000-02-11T00:00:00.000Z', 'EndingDateTime': '2000-02-21T23:59:59.000Z'}}
Size(MB): 4.87553
Data: ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/SRTMGL1.003/N46W097.SRTMGL1.hgt/N46W097.SRTMGL1.hgt.zip'], Collection: {'ShortName': 'SRTMGL1', 'Version': '003'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -98.00027778, 'EastBoundingCoordinate': -96.99972222, 'NorthBoundingCoordinate': 47.00027778, 'SouthBoundingCoordinate': 45.99972222}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2000-02-11T00:00:00.000Z', 'EndingDateTime': '2000-0

In [10]:
## Creating a pattern for selecting srtm tiles based on the grassland gdf bounds

# Extract bounding box from shey_gdf
xmin, ymin, xmax, ymax = shey_gdf.total_bounds

# Generate SRTM tile names based on integer degree tiles
latitudes = range(int(ymin), int(ymax) + 1)
longitudes = range(int(xmin), int(xmax) + 1)

# Create SRTM file patterns
srtm_p_pattern_list = []
for lat in latitudes:
    for lon in longitudes:
        lat_prefix = f"N{lat}" if lat >= 0 else f"S{abs(lat)}"
        lon_prefix = f"E{lon}" if lon >= 0 else f"W{abs(lon)}"
        srtm_p_pattern_list.append(os.path.join(elevation_dir, f"{lat_prefix}{lon_prefix}.hgt.zip"))

# # Use glob to find matching files
# srtm_p_pattern = [glob(pattern) for pattern in srtm_p_pattern_list]
# srtm_p_pattern = [item for sublist in srtm_p_pattern for item in sublist]  # Flatten list

# print("Matching SRTM files:", srtm_p_pattern)


In [11]:
srtm_p_pattern_list

['C:\\Users\\moenc\\earth-analytics\\data\\srtm\\N46W97.hgt.zip',
 'C:\\Users\\moenc\\earth-analytics\\data\\srtm\\N46W96.hgt.zip']

In [12]:
print(os.listdir(elevation_dir))

['N36W103.SRTMGL1.hgt.zip', 'N36W104.SRTMGL1.hgt.zip', 'N36W105.SRTMGL1.hgt.zip', 'N37W103.SRTMGL1.hgt.zip', 'N37W104.SRTMGL1.hgt.zip', 'N37W105.SRTMGL1.hgt.zip', 'N40W104.SRTMGL1.hgt.zip', 'N40W105.SRTMGL1.hgt.zip', 'N41W104.SRTMGL1.hgt.zip', 'N41W105.SRTMGL1.hgt.zip', 'N43W102.SRTMGL1.hgt.zip', 'N43W103.SRTMGL1.hgt.zip', 'N43W104.SRTMGL1.hgt.zip', 'N43W105.SRTMGL1.hgt.zip', 'N46W103.hgt', 'N46W103.SRTMGL1.hgt.zip', 'N46W104.hgt', 'N46W104.SRTMGL1.hgt.zip', 'N46W105.hgt', 'N46W105.SRTMGL1.hgt.zip', 'N47W103.SRTMGL1.hgt.zip', 'N47W104.SRTMGL1.hgt.zip', 'N47W105.SRTMGL1.hgt.zip', 'N48W103.SRTMGL1.hgt.zip', 'N48W104.SRTMGL1.hgt.zip', 'N48W105.SRTMGL1.hgt.zip']


In [32]:
## Note: Edits needed! Downloads a single zip file.

### zip path -- we know we want the "08" file because the watershed code starts with "08"
zip_path = os.path.join(elevation_dir, "C:\\Users\\moenc\\earth-analytics\\data\\srtm\\N46W97.hgt.zip")
# Download the zip file once
if not os.path.exists(zip_path):
    ### query with the url
    response = requests.get(shey_url)
    ### check if response was successful
    if response.status_code == 200:
        ### save the zip file
        with open(zip_path, 'wb') as f:
            f.write(response.content)
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")
        exit()

NameError: name 'requests' is not defined

In [ ]:
# # Define a pattern to identify DEM tiles associated with curl National Grassland.
# srtm_p_pattern = [
#     os.path.join(elevation_dir, 'N46*hgt.zip'),
#     # os.path.join(elevation_dir, 'N42*hgt.zip'),
# ]
# bounds_p = tuple(shey_gdf.total_bounds)
# buffer = 0.25
# xmin, ymin, xmax, ymax = bounds_p
# bounds_buffer = (xmin-buffer, ymin-buffer, xmax+buffer, ymax+buffer)

# # compile srtm files into list
# all_files = []
# for pattern in srtm_p_pattern:
#     all_files.extend(glob(pattern))

# if not all_files:
#     srtm_p_results = earthaccess.search_data(
#         short_name="SRTMGL1",
#         bounding_box=bounds_p
#     )
#     srtm_p_results = earthaccess.download(srtm_p_results, elevation_dir)

# if not glob(srtm_p_pattern):
#     srtm_p_results = earthaccess.search_data(
#         short_name = "SRTMGL1",
#         bounding_box=bounds_buffer    
#     )
#     srtm_p_results = earthaccess.download(srtm_p_results, elevation_dir)

TypeError: expected str, bytes or os.PathLike object, not list

In [ ]:
# all_files

NameError: name 'all_files' is not defined

In [26]:
import zipfile # extract the zipped files

# Path to the downloaded ZIP file
zip_path = 'C:/Users/moenc/earth-analytics/data/srtm/N46W96.hgt.zip'

# Folder where you want to extract the contents
extract_path = elevation_dir

# Open the ZIP file in read mode
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Extract all contents to the specified folder
    zip_ref.extractall(extract_path)

print("Unzipping complete")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/moenc/earth-analytics/data/srtm/N46W96.hgt.zip'

In [ ]:
# # Print the DEM for curl natl. Grassland
# srtm_p_da_list=[]
# for srtm_p_path in glob(srtm_c_pattern):
#     tile_da = rxr.open_rasterio(srtm_p_path, mask_and_scale=True).squeeze()
#     cropped_da = tile_da.rio.clip_box(*bounds_buffer)
#     srtm_p_da_list.append(cropped_da)
    
# srtm_p_da = rxrmerge.merge_arrays(srtm_p_da_list)
# srtm_p_da.plot(cmap='terrain')
# curl_gdf.boundary.plot(ax=plt.gca(), color='black')

# Fix the second code block.
srtm_p_da_list = []
for pattern in srtm_p_pattern:
    for srtm_p_path in glob(pattern):
        tile_da = rxr.open_rasterio(srtm_p_path, mask_and_scale=True).squeeze()
        cropped_da = tile_da.rio.clip_box(*bounds_buffer)
        srtm_p_da_list.append(cropped_da)

srtm_p_da = rxrmerge.merge_arrays(srtm_p_da_list)
srtm_p_da.plot(cmap='terrain')
shey_gdf.boundary.plot(ax=plt.gca(), color='white')

NoDataInBounds: No data found in bounds.

In [ ]:
%store srtm_p_da

UsageError: Unknown variable 'srtm_c_da'


In [ ]:
srtm_p_results = earthaccess.search_data(
    short_name="SRTMGL1",
    bounding_box=bounds_buffer    
)
print(srtm_p_results)  # See if any results are found


[Collection: {'ShortName': 'SRTMGL1', 'Version': '003'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -113.00027778, 'EastBoundingCoordinate': -111.99972222, 'NorthBoundingCoordinate': 43.00027778, 'SouthBoundingCoordinate': 41.99972222}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2000-02-11T00:00:00.000Z', 'EndingDateTime': '2000-02-21T23:59:59.000Z'}}
Size(MB): 11.1593
Data: ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/SRTMGL1.003/N42W113.SRTMGL1.hgt/N42W113.SRTMGL1.hgt.zip'], Collection: {'ShortName': 'SRTMGL1', 'Version': '003'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -113.00027778, 'EastBoundingCoordinate': -111.99972222, 'NorthBoundingCoordinate': 42.00027778, 'SouthBoundingCoordinate': 40.99972222}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2000-02-11T00:00:00.000Z', 'EndingDateTime': '20

In [ ]:
## Creating a pattern for selecting srtm tiles based on the grassland gdf bounds

# Extract bounding box from shey_gdf
xmin, ymin, xmax, ymax = shey_gdf.total_bounds

# Generate SRTM tile names based on integer degree tiles
latitudes = range(int(ymin), int(ymax) + 1)
longitudes = range(int(xmin), int(xmax) + 1)

# Create SRTM file patterns
srtm_c_pattern_list = []
for lat in latitudes:
    for lon in longitudes:
        lat_prefix = f"N{lat}" if lat >= 0 else f"S{abs(lat)}"
        lon_prefix = f"E{lon}" if lon >= 0 else f"W{abs(lon)}"
        srtm_c_pattern_list.append(os.path.join(elevation_dir, f"{lat_prefix}{lon_prefix}.hgt.zip"))

# # Use glob to find matching files
# srtm_p_pattern = [glob(pattern) for pattern in srtm_p_pattern_list]
# srtm_p_pattern = [item for sublist in srtm_p_pattern for item in sublist]  # Flatten list

# print("Matching SRTM files:", srtm_p_pattern)

In [ ]:
# define a pattern which identifies the DEM's belongning to shey National Grasslands

srtm_p_pattern = os.path.join(elevation_dir, 'N*hgt.zip')
bounds_p = tuple(shey_gdf.total_bounds)
if not glob(srtm_p_pattern):
    srtm_p_results = earthaccess.search_data(
        short_name = "SRTMGL1",
        bounding_box=bounds_p    
    )
    srtm_p_results = earthaccess.download(srtm_p_results, elevation_dir)

In [ ]:
srtm_p_pattern

'C:\\Users\\moenc\\earth-analytics\\data\\srtm\\N*hgt.zip'

In [ ]:
bounds_p

(np.float64(-97.46880119000002),
 np.float64(46.09533527999997),
 np.float64(-96.93853933000003),
 np.float64(46.57203363000002))

In [ ]:
# # Print the DEM for shey natl. Grassland
# srtm_p_da_list=[]
# for srtm_p_path in glob(srtm_p_pattern):
#     tile_da = rxr.open_rasterio(srtm_p_path, mask_and_scale=True).squeeze()
#     try:
#         cropped_da = tile_da.rio.clip_box(*bounds_p)
#     except: 
#         continue
#     srtm_p_da_list.append(cropped_da)
    
# srtm_p_da = rxrmerge.merge_arrays(srtm_p_da_list)
# srtm_p_da.plot(cmap='terrain')
# shey_gdf.boundary.plot(ax=plt.gca(), color='black')

In [ ]:
# reproject the curl DEM into utm 13 N crs
utm_13n_epsg = 32613
srtm_p_proj_da = srtm_p_da.rio.reproject(utm_13n_epsg)
srtm_p_proj_da = srtm_da.to_crs()
srtm_p_proj_da.plot()

NameError: name 'srtm_c_da' is not defined

In [ ]:
# Reproject so units are in meters
utm13_epsg = 32613
srtm_p_proj_da = srtm_p_da.rio.reproject(utm13_epsg)
shey_proj_gdf = shey_gdf.to_crs(utm_13n_epsg)
bounds_proj = tuple(shey_proj_da.total_bounds)

# Calculate slope
slope_full_da = xrspatial.slope(srtm_p_proj_da)
# slope_da = slope_full_da.rio.clip_box(*bounds_proj)
slope_da = slope_full_da.rio.clip(shey_proj_gdf.geometry)

# Plot slope, with curl bounds overlay
slope_p_da.plot(cmap='terrain')
shey_proj_gdf.boundary.plot(ax=plt.gca(), color='white', linewidth=.5)
plt.show()

NameError: name 'lmis_proj_da' is not defined